# Deploy and verify the deployment

Steps:
- Deploy model using docker image
- Run inference to check if its up and running

In [3]:
!pip install kfp --upgrade -q

In [4]:
import kfp
from kfp import dsl

In [5]:
client = kfp.Client()

## Deploy Model

In [21]:
def deploy(image_url:str, deployment_name:str, predictor_name:str):
    import subprocess
    
    with open("/tmp/manifest.yaml", "w") as f:
        manifest = f"""
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: {deployment_name}
spec:
  name: {deployment_name}
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - name: classifier
          image: {image_url}
          securityContext:
            allowPrivilegeEscalation: false
            runAsUser: 0
    graph:
      name: classifier
    name: {predictor_name}
    replicas: 1
    labels:
      sidecar.istio.io/inject: "false"
        """
        print(manifest)
        f.write(manifest)
    
    result = subprocess.call(['kubectl', 'apply', '-f', '/tmp/manifest.yaml', '-n', 'admin'])
    assert result == 0

In [22]:
deploy_op = kfp.components.create_component_from_func(
        func=deploy,
        output_component_file='deploy-component.yaml', # This is optional. It saves the component spec for future use.
        base_image='bponieckiklotz/seldon-deploy:0.1',
        packages_to_install=[])

## Check response

In [27]:
def validate(deployment_name:str, predictor_name:str, namespace:str, image_url:str, max_retries:int=20, sleep_time:int=30):
    import requests
    import uuid
    import time
    import json
    body = {
        "data": {
            "ndarray": [str(uuid.uuid4()), "Hi, how are you?"]
        }
    }
    deployed = False
    count = max_retries
    while not deployed:
        try:
            res = requests.post(f"http://{deployment_name}-{predictor_name}.{namespace}.svc.cluster.local:8000/api/v0.1/predictions", json=body)
            print(res.status_code, res.text)
            if res.status_code == 200 and json.loads(res.text).get("meta").get("requestPath").get("classifier") == image_url:
                deployed = True
            else:
                count -= 1
                print("Sleeping 1 sec")
                time.sleep(sleep_time)
        except Exception as e:
            print(e)
            count -= 1
            print(f"Sleeping {sleep_time} sec. (except)")
            time.sleep(sleep_time)
        


In [28]:
validate_op = kfp.components.create_component_from_func(
        func=validate,
        output_component_file='validate-component.yaml', # This is optional. It saves the component spec for future use.
        base_image='bponieckiklotz/seldon-deploy:0.1',
        packages_to_install=['requests'])

## Create pipeline

In [29]:
from kubernetes.client.models import V1EnvVar
from kfp.onprem import use_k8s_secret

@dsl.pipeline(
    name="chatbot_pipeline",
    description="Chatbot pipeline",
)
def chatbot_deploy_pipeline(image_url, deployment_name="chatbot", predictor_name="default", namespace="admin"):
    deploy_task = deploy_op(image_url=image_url, 
                            deployment_name=deployment_name, 
                            predictor_name=predictor_name)
    validate_task = validate_op(deployment_name=deployment_name, 
                                predictor_name=predictor_name, 
                                namespace=namespace,
                                image_url=image_url
                               ).after(deploy_task)


In [30]:
client.create_run_from_pipeline_func(
    chatbot_deploy_pipeline,
    arguments={
        "image_url": "bponieckiklotz/jellyfish.chatbot:dev@sha256:a1ce5fcdc31e3c393eb47e18245bebc789aa6879f54611471c0a57f0a440b2e4",
        "deployment_name": "chatbot",
        "predictor_name": "default",
        "namespace": "admin",
        
    })

RunPipelineResult(run_id=c8557831-b9c0-4f01-8be7-1d135df8b47b)